In [21]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1)) # 参考15中的类，我们已经默认给参数初始化了

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [22]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [23]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [24]:
# 如果一个 Tensor 是 Parameter ， 其会自动被添加到模型的参数列表里
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():    # weight2 不是parameter 因此不会被添加到参数列表中
    print(name)

weight1


In [25]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[-0.4428,  0.1496,  0.4320,  0.4944],
        [-0.4319, -0.0840, -0.3474, -0.3167],
        [-0.4317,  0.1408, -0.3235,  0.4470]])
None
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


# 初始化模型参数

In [26]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[-0.0031, -0.0162,  0.0083,  0.0127],
        [-0.0074, -0.0112,  0.0202, -0.0040],
        [ 0.0059, -0.0094, -0.0092, -0.0168]])
2.weight tensor([[ 0.0052,  0.0032, -0.0058]])


In [27]:
# 以常数来初始化权重参数
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


# 自定义初始化方法

In [28]:
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():   # 在变化过程中，不计算入梯度中
        return tensor.normal_(mean, std)

In [29]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()   # 绝对值都大于等于5
        
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[ 7.1418,  0.0000,  8.0323, -0.0000],
        [ 6.9340,  5.6689,  9.7114, -0.0000],
        [-9.7034,  0.0000,  6.7096, -6.6080]])
2.weight tensor([[-0.0000, -8.2736, -9.6270]])


# 共享模型参数

In [30]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)     # 在内存中，这两个线性层实际上是一个对象 共享的梯度参数是累加的
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [31]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
